In [1]:
from mmnl import MMNL
import numpy as np
import pickle
import pandas as pd


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Authorize Google Drive to upload files to the drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
def load_data(path):
    dat = np.load(path)
    X = dat[:, :-1]
    Y = np.reshape(dat[:, -1], (-1, 1))
    return X, Y
X, Y = load_data('data/data.npy')

Load data

In [3]:
%%time
infile = open('./data/500_QMC_dgp.p', 'rb')
big_dict = pickle.load(infile)
Y_dgp = big_dict['theta: [ 1.5  1.  -1.1  0.8  0.1  1.2]']
print(Y_dgp.shape)

(2798, 500)
Wall time: 32.2 ms


In [ ]:
%%time
print(Y_dgp[:,0])
result = []
draws = 25
start = 50
end = 100
method = 'QMC'
for i in range(start,end, 10):
    for j in range(i,i+10):
        print('iteration: %i' %(i+1))
        qmc = MMNL(X,Y_dgp[:,j],draws,3,method=method)
        solution = qmc.solver()
        result.append(solution)
    pickle.dump( result, open( "./resultaten/%i-%i_%s(%i)_results.p" %(i,i+10,method,draws), "wb" ) )
#     f = drive.CreateFile()
#     f.SetContentFile("./resultaten/%i-%i_%s(%i)_results.p" %(i,i+10,method,draws))
#     f.Upload()

[1. 1. 1. ... 3. 3. 3.]
iteration: 51


C:\Users\niels\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:502: RuntimeWarning: Method Nelder-Mead does not use gradient information (jac).
  RuntimeWarning)


 final_simplex: (array([[ 0.8992118 ,  1.11518763, -2.93969227, -0.19730697, -0.23405917,
         0.53121242],
       [ 0.89921349,  1.11518981, -2.93969828, -0.19727395, -0.23412972,
         0.53123565],
       [ 0.89921418,  1.11518892, -2.93969639, -0.19738804, -0.23412667,
         0.53123005],
       [ 0.89921249,  1.11518971, -2.93969754, -0.19721371, -0.23409331,
         0.53122146],
       [ 0.89920981,  1.11518554, -2.93968673, -0.19727326, -0.23407166,
         0.53122044],
       [ 0.89921397,  1.11519066, -2.93970014, -0.19728937, -0.234058  ,
         0.531209  ],
       [ 0.89921053,  1.11518538, -2.93968658, -0.19729851, -0.23414603,
         0.53119323]]), array([1680.86786815, 1680.86786815, 1680.86786815, 1680.86786815,
       1680.86786815, 1680.86786815, 1680.86786815]))
           fun: 1680.867868145464
       message: 'Optimization terminated successfully.'
          nfev: 770
           nit: 490
        status: 0
       success: True
             x: array([ 0.

In [10]:
x_est = np.array(result)
x_est.shape

(100, 6)

In [11]:
np.mean(x_est, axis=0)

array([ 4.58777441,  2.65487859, -7.12572568,  7.45172249,  2.22983804,
        5.23629935])